## [실습1] 시퀀스-투-시퀀스 학습

영어-스페인어 기계 번역과 영어-한국어 기계 번역을 작은 데이터셋을 이용하여 훈련시켜본다.

### 영어-스페인어 기계 번역

**텍스트 데이터셋 다운로드**

영어와 스페인어 텍스트가 담긴 압축 파일을 다운로드 한 후에 압축을 풀면
"spa.txt" 파일이 생성된다.

In [6]:
!wget https://www.manythings.org/anki/spa-eng.zip
!unzip -q spa-eng.zip

--2025-06-02 15:10:22--  https://www.manythings.org/anki/spa-eng.zip
www.manythings.org (www.manythings.org) 해석 중... 173.254.30.110
다음으로 연결 중: www.manythings.org (www.manythings.org)|173.254.30.110|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 5420295 (5.2M) [application/zip]
저장 위치: `spa-eng.zip'

spa-eng.zip         100%[===================>]   5.17M  3.24MB/s    /  1.6s    

2025-06-02 15:10:24 (3.24 MB/s) - `spa-eng.zip' 저장함 [5420295/5420295]



In [15]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
!brew install wget

==> Downloading https://ghcr.io/v2/homebrew/core/wget/manifests/1.25.0
######################################################################### 100.0%
==> Fetching dependencies for wget: libidn2
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/manifests/2.3.8
######################################################################### 100.0%
==> Fetching libidn2
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/blobs/sha256:d0d933dad3
######################################################################### 100.0%
==> Fetching wget
==> Downloading https://ghcr.io/v2/homebrew/core/wget/blobs/sha256:7fce09705a52a
######################################################################### 100.0%
==> Installing dependencies for wget: libidn2
==> Installing wget dependency: libidn2
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/manifests/2.3.8
Already downloaded: /Users/jamie/Library/Caches/Homebrew/downloads/f30f50fbde4bff9a71de54d684e482d7da3432656d680b97441163c6e5665

"spa.txt" 파일은 각각의 줄은 아래와 같이 영어 텍스트, 스페인어 텍스트, 기타 정보가 탭(tab) 키로 구분되어 있다.

```
Finally, it's Friday.	Al fin es viernes.	CC-BY 2.0 (France) Attribution: tatoeba.org #433868 (CK) & #1427385 (marcelostockle)
```

아래 코드는 "spa.txt"에 포함된 각 줄의 내용을 항목으로 갖는 리스트인 `text_pairs`를 생성한다.
단 각 항목은 (영어 텍스트, 스페인어 텍스트)로 구성된 튜플이며, 각각의 줄에 포함된 기타 정보는 버린다.
또한 스페인어 텍스트의 처음과 끝에 각각 `'[start] '` 와 `' [end]'`를 추가한다.

In [7]:
text_file = "spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    english, spanish, _ = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

`text_pairs`에 포함된 임의의 항목을 확인하면 다음과 같다.

In [8]:
import random

print(random.choice(text_pairs))

('You should turn off your cell phone.', '[start] Debería apagar su teléfono móvil. [end]')


아래 코드는 텍스트를 무작위 섞은 다음
70 대 15 대 15의 비율로 훈련 텍스트셋, 검증 텍스트셋, 테스트 텍스트셋으로 나눈다.

In [9]:
random.shuffle(text_pairs)

# 검증셋 크기: 전체 데이터셋의 15%
num_val_samples = int(0.15 * len(text_pairs))
# 훈련셋 크기: 전체 데이터셋의 70%
num_train_samples = len(text_pairs) - 2 * num_val_samples

# 훈련 텍스트셋
train_pairs = text_pairs[:num_train_samples]
# 검증 텍스트셋
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
# 테스트 텍스트셋
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**영어/스페인어 텍스트 벡터화**

자연어로 구성된 훈련 텍스트 데이터셋을 대상으로 어휘 인덱스를 생성한 후에 텍스트 벡터화를 진행한다.
먼저 영어 어휘집을 생성한다.
생성되는 어휘 벡터의 길이를 20으로 지정한다.

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

vocab_size = 15000
sequence_length = 20

# 번역 대상 언어(예를 들어 영어) 텍스트 데이터셋 벡터화 층
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# 영어 텍스트만 추출
train_english_texts = [pair[0] for pair in train_pairs]
# 영어 어휘집 생성
source_vectorization.adapt(train_english_texts)

2025-06-02 15:14:38.333133: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-06-02 15:14:38.333172: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-06-02 15:14:38.333178: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-06-02 15:14:38.333239: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-02 15:14:38.333284: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-06-02 15:14:56.167356: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


스페인어 텍스트 벡터화는 영어와는 다른 표준화 방식을 사용한다.

- 영어에는 없는 `'¿'` 기호도 표준화 과정에서 삭제
- 반면에 `'['`와 `']'`는 표준화 과정에서 제거되지 않도록 지정

또한 생성되는 어휘 벡터의 길이를 21로 지정한다.
그러면 0번 인덱스부터 19번 인덱스까지는 입력값으로,
1번 인덱스부터 20번 인덱스까지는 타깃으로 지정할 수 있다.

In [11]:
import string
import re

# 마침표 기호 목록에 "¿" 추가. 즉 표준화과정에서 삭제 대상으로 지정.
strip_chars = string.punctuation + "¿"
# 마침표 기호 목록으로부터 "[" 와 "]" 제거. 즉, 표준화 대상에서 삭제하지 않도록 함.
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

# 새로운 표준화 함수 선언
# 소문자로 변환한 후에 strip_chars 에 포함된 모든 기호 삭제
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

# 번역 언어 (예를 들어 스페인어) 텍스트 데이터셋 벡터화 층
# 벡터의 길이를 20이 아닌 21로 지정. 입력값과 타깃을 구분하기 위해 필요함.
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

# 스페인어 텍스트만 추출
train_spanish_texts = [pair[1] for pair in train_pairs]
# 스페인어 어휘집 생성
target_vectorization.adapt(train_spanish_texts)

훈련 텍스트셋, 검증 텍스트셋, 테스트 텍스트셋을 모두 어휘 인덱스를 이용하여 벡터화 한다.

아래 코드는 생성된 영어와 스페인어 어휘 인덱스를 이용하여 각각의 텍스트 데이터셋을
벡터화 한 다음에 아래 모양의 튜플로 구성된 훈련셋, 검증셋, 테스트셋을 생성한다.

- 튜플의 첫째 항목: 영어 입력 배치와 스페인어 입력 배치로 구성된 사전. 모델의 입력값으로 사용.
- 튜플의 둘째 항목: 타깃 배치. 모델 훈련의 타깃으로 사용.

```
({"english": 영어 입력 배치, "spanish": 스페인어 입력 배치}, 타깃 배치)
```

- `format_dataset()` 함수
    - 인자: 영어 텍스트 배치와 스페인어 텍스트 배치
    - 반환값: 앞서 언급한 모양의 사전
    
- `make_dataset()` 함수
    - 인자: `(영어 텍스트, 스페인어 텍스트)` 모양의 튜플로 구성된 자연어 텍스트 데이터셋\
    - 반환값: 지정된 배치 크기로 묶은 배치들에 대해 `format_dataset()` 함수를 적용하여
        생성된 `Dataset` 자료형의 데이터셋. 배치 단위로 묶여 있음.
        - `dataset.shuffle(2048).prefetch(16).cache()`: 대용량 데이터셋을 배치 단위로
            빠르게 불러오기 위해 사용함.        

In [12]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({"english": eng, "spanish": spa[:, :-1]}, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)

    dataset = dataset.map(format_dataset)

    return dataset.shuffle(2048).prefetch(16).cache()

# 훈련셋
train_ds = make_dataset(train_pairs)
# 검증셋
val_ds = make_dataset(val_pairs)
# 테스트셋
test_ds = make_dataset(test_pairs)

예를 들어 훈련셋의 첫째 배치의 모양을 확인하면 다음과 같다.

- 영어 입력 배치: 길이가 20인 64개의 벡터로 구성. 즉, 20 개의 단어로 구성된 영어 텍스트 64개로 구성됨.
- 스페인어 입력 배치: 길이가 20인 64개의 벡터로 구성. 즉, 20 개의 단어로 구성된 스페인어 텍스트 64개로 구성됨.
- 타깃 배치: 길이가 20인 64개의 벡터로 구성. 즉, 20 개의 단어로 구성된 스페인어 텍스트 64개로 구성됨.

In [14]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2025-06-02 15:16:50.909680: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


아래 코드는 첫째 샘플의 영어 벡터, 스페인어 벡터, 타깃을 보여준다.
스페인어 입력 벡터 샘플의 0번 인덱스에 위치한 정수 2가 `'[start]'`에 해당하는 값이다.
스페인어 타깃 벡터 샘플은 그 값을 제외한 벡터로 시작함을 확인할 수 있다.
또한 정수 3은 `'[end]'`에 해당하는 값이며, 문장의 끝을 가리키기에
스페인어 타깃 벡터 샘플에 새로운 단어에 해당하는 인덱스를 추가하지 않고 대신 0 패딩이 하나 더 추가되었다.

In [18]:
for inputs, targets in train_ds.take(1):
    print(f"영어 입력 벡터 샘플: {inputs['english'][0]}")
    print(f"스페인어 입력 벡터 샘플: {inputs['spanish'][0]}")
    print(f"스페인어 타깃 샘플: {targets[0]}")

영어 입력 벡터 샘플: [ 64  54   9  29 118   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
스페인어 입력 벡터 샘플: [   2  170 1199    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
스페인어 타깃 샘플: [ 170 1199    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


2025-06-02 15:17:31.581907: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### 트랜스포머 디코더

트랜스포머 디코더를 하나의 층으로 구현하면 다음과 같다.
생성자의 인자는 다음과 같다.

- `embed_dim`: 예를 들어 `embed_dim=256`은 단어 임베딩 `(600, 256)` 모양의 샘플 생성
- `dense_dim`: 밀집층에서 사용되는 유닛<font size='2'>unit</font> 개수
- `num_heads`: 헤드<font size='2'>head</font> 개수

`get_causal_attention_mask()` 메서드는 스페인어 입력 텍스트에 대한 마스크를 지정할 때 활용되지만
여기서는 마스크를 사용하지 않는다.

순전파를 담당하는 `call()` 메서드는 두 개의 어텐션 층을 사용한다.
입력값으로는 스페인어 텍스트 배치 데이터셋과
트랜스포머 디코더의 출력값으로 셀프 어텐션이 적용되어 변환된 영어 텍스트 배치 데이터셋이 사용된다.

- `attention_1`: 스페인어 텍스트 입력값에 대해 셀프 어텐션 적용
- `attention_2`: `attention_1` 의 출력값을 query로, 트랜스포머 인코더의 출력값을 key와 value로 사용해서 어텐션 적용.

최종적으로 두 개의 밀집층을 통과시킨다.
또한 하나의 블록을 통과시킬 때마다 잔차연결과 층정규화를 진행한다.

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.not_equal(inputs, 0)  # tf.not_equal 사용

    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

In [26]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        
        # 셀프 어텐션 적용
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        
        # 피드포워드 네트워크 적용
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config

In [27]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        # 마스크 활용
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        # 셀프 어텐션 적용: 번역 언어(예를 들어 스페인어) 입력값 대상
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        # 셀프 어텐션이 적용된 (예를 들어 스페인어) 입력 텍스트를 query로
        # 셀프 어텐션이 적용된 번역 대상 (예를 들어 영어) 입력 텍스트를 key와 value로
        # 지정하여 어텐션 적용
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)

        proj_output = self.dense_proj(attention_output_2)

        return self.layernorm_3(attention_output_2 + proj_output)

### 기계 번역 모델

모델의 입력값은 앞서 설명한 대로 예를 들어 일정 길이로 단어 벡터화된 영어 텍스트 데이터셋과
스페인어 텍스트 데이터셋의 튜플이다.
스페인어 텍스트는 모두 `[start]` 로 시작하도록 전처리되어 있다.

모델의 출력값은 예를 들어 출력 스페인어 텍스트로 지정될 단어들에 대한 위치별 확률값을 계산한다.
아래 코드에서는 스페인어 텍스트에 포함될 20 개 단어들의 후보를 위치별로 확률값으로 계산한다.
예를 들어 출력 텍스트의 i-번 인덱스에 위치할 단어의 확률값을 계산하기 위해
어휘집에 포함된 15,000 개 단어를 대상으로 각각의 단어가 해당 위치에 자리할 확률을
소프트맥스 함수를 이용하여 계산한다.

In [28]:
sequence_length = 20 # 텍스트의 단어수
vocab_size = 15000 # 어휘집 크기
embed_dim = 256    # 단어 임베딩 크기
dense_dim = 2048   # 밀집층 유닛수
num_heads = 8      # 어텐션 헤드수

# 트랜스포머 인코더 활용

# 첫째 입력값: 예를 들어 영어 텍스트셋
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 트랜스포머 디코더 활용

# 둘째 입력값: 예를 들어 스페인어 텍스트셋
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**모델 훈련과 활용**

모델의 최종 출력값이 소프트맥스를 사용하여
`(20, 15000)` 모양을 갖는 반면에
타깃셋은 20 개의 어휘 인덱스로 구성된 벡터로 구성되기에
`categorical_crossentropy` 가 아닌 `sparse_categorical_crossentropy`를
손실함수로 지정한다.
그러면 20개 단어 각각에 대해 가장 높은 확률을 갖는 (어휘) 인덱스에 해당하는 단어가
15,000 개 중에 선택되어 타깃 단어와 비교된다.

In [29]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1549/1549 [==============================] - 1247s 802ms/step - loss: 4.0514 - accuracy: 0.4343 - val_loss: 3.2309 - val_accuracy: 0.5222
Epoch 2/30
1549/1549 [==============================] - 1184s 764ms/step - loss: 3.2023 - accuracy: 0.5484 - val_loss: 2.8252 - val_accuracy: 0.5797
Epoch 3/30
1549/1549 [==============================] - 1246s 804ms/step - loss: 2.8465 - accuracy: 0.5943 - val_loss: 2.6281 - val_accuracy: 0.6173
Epoch 4/30
1549/1549 [==============================] - 1170s 755ms/step - loss: 2.6730 - accuracy: 0.6233 - val_loss: 2.5349 - val_accuracy: 0.6341
Epoch 5/30
1549/1549 [==============================] - 1163s 751ms/step - loss: 2.5849 - accuracy: 0.6414 - val_loss: 2.5037 - val_accuracy: 0.6425
Epoch 6/30
 422/1549 [=======>......................] - ETA: 13:54 - loss: 2.5424 - accuracy: 0.6504

KeyboardInterrupt: 

아래 `decode_sequence()`는 함수는 영어 텍스트가 하나 입력되면
앞서 훈련된 트랜스포머 모델을 이용하여 지정된 길이인 20 개의 단어로
구성된 스페인어 텍스트를 생성한다.

함수 본문에 포함된 `for` 반복문은
**트랜스포머 모델 활용** 부분에서 설명한 방식 그대로
`[start]`로만 구성된 텍스트로 시작해서
계속해서 텍스트에 추가할 단어를 하나씩 선택해서 이어가는 과정을
`[end]` 키워드가 나올 때까지 반복한다.
단, 반복횟수는 20으로 제한한다.

In [ ]:
import numpy as np

# 어휘집 확인
spa_vocab = target_vectorization.get_vocabulary()
# (단어 인덱스, 단어)로 구성된 사전 지정
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
# 텍스트에 포함되는 단어수
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    # 기계 번역 시작
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        # 트랜스포머 모델 적용
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])

        # i-번째 단어로 사용될 어휘 인덱스 확인
        sampled_token_index = np.argmax(predictions[0, i, :])
        # i-번째 단어 확인
        sampled_token = spa_index_lookup[sampled_token_index]
        # 스페인어 입력 텍스트에 i-번째 단어로 추가
        decoded_sentence += " " + sampled_token
        # 기계 번역 종료 조건 확인
        if sampled_token == "[end]":
            break

    return decoded_sentence

아래 코드는 `decode_sequence()` 함수를 이용하여
무작위로 5개의 영어 텍스트를 선택하여 기계 번역한 결과이다.

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))